In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data = pd.read_csv('/content/drive/MyDrive/Social Media Group /Data/topic_modeling_test.csv')

In [4]:
r = pd.read_csv('/content/drive/MyDrive/Social Media Group /Data/Ratios.csv')

# Fill missing tickers

In [5]:
ticker = data['tic'].drop_duplicates().tolist()

In [6]:
# build a permno --> TICKER dict to handle missing TICKER
tmp = r.dropna(subset=['TICKER']).drop_duplicates(subset=['permno'])
permno_TICKER = dict()
for i in tqdm(range(len(tmp))):
    permno = tmp['permno'].iloc[i]
    TICKER = tmp['TICKER'].iloc[i]
    permno_TICKER[permno] = TICKER

100%|██████████| 1713/1713 [00:00<00:00, 49025.89it/s]


In [7]:
# filling missing tickers
for i in tqdm(list(r[r['TICKER'].isna()].index)):
    if r['permno'].iloc[i] in permno_TICKER:
        r['TICKER'].iloc[i] = permno_TICKER[r['permno'].iloc[i]]

  0%|          | 0/434 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
100%|██████████| 434/434 [00:00<00:00, 2653.40it/s]


In [8]:
# drop na ticker rows
r = r.dropna(subset=['TICKER']).reset_index().drop(columns='index')

# Ratios

In [9]:
# filter out datapoints that are not in our result dataset
r['yn'] = r['TICKER'].apply(lambda x: True if x in ticker else False)

In [10]:
r = r[r['yn'] == True]

In [11]:
r

,permno,adate,qdate,public_date,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,pcf,dpr,ptb,PEG_trailing,divyield,TICKER,yn
0,10026,20150930.0,20160630,20160930,34.285,0.299,13.794,29.558,29.780,29.780,29.780,2.245,20.587,0.372,3.345,4.935,1.31%,JJSF,True
1,10026,20150930.0,20160630,20161031,35.157,0.299,13.794,30.310,30.538,30.538,30.538,2.303,21.111,0.372,3.431,5.060,1.28%,JJSF,True
2,10026,20160930.0,20160930,20161130,33.713,0.309,14.600,29.835,29.983,29.983,29.983,2.285,18.715,0.383,3.306,4.930,1.28%,JJSF,True
3,10026,20160930.0,20160930,20161231,37.072,0.309,14.600,32.784,32.946,32.946,32.946,2.513,20.579,0.383,3.636,5.417,1.26%,JJSF,True
4,10026,20160930.0,20160930,20170131,35.467,0.309,14.600,31.344,31.499,31.499,31.499,2.404,19.688,0.383,3.478,5.179,1.32%,JJSF,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63055,93426,20181231.0,20190630,20190930,53.946,0.430,9.662,15.228,15.299,17.891,17.891,1.365,9.084,0.000,1.732,NaN,NaN,VPG,True
63056,93426,20181231.0,20190630,20191031,56.105,0.430,9.662,15.837,15.911,18.607,18.607,1.419,9.448,0.000,1.801,NaN,NaN,VPG,True
63057,93426,20181231.0,20190930,20191130,52.343,0.539,10.103,18.016,18.206,21.506,21.506,1.474,10.228,0.000,1.801,NaN,NaN,VPG,True
63058,93426,20181231.0,20190930,20191231,51.720,0.539,10.103,17.801,17.989,21.250,21.250,1.456,10.106,0.000,1.780,NaN,NaN,VPG,True


In [12]:
# set the threshold to 1000 null datapoints
drop_columns = list()
for index in range(4,len(r.columns)):
    if r.iloc[:,index].isnull().sum() > 3000:
        drop_columns.append(r.iloc[:,index].name)


In [13]:
# no. of tickers
len(r.drop_duplicates(subset=['TICKER']))

1579

In [14]:
len(drop_columns)

3

In [15]:
len(r.columns)

19

In [16]:
r.head()

,permno,adate,qdate,public_date,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,pcf,dpr,ptb,PEG_trailing,divyield,TICKER,yn
0,10026,20150930.0,20160630,20160930,34.285,0.299,13.794,29.558,29.780,29.780,29.780,2.245,20.587,0.372,3.345,4.935,1.31%,JJSF,True
1,10026,20150930.0,20160630,20161031,35.157,0.299,13.794,30.310,30.538,30.538,30.538,2.303,21.111,0.372,3.431,5.060,1.28%,JJSF,True
2,10026,20160930.0,20160930,20161130,33.713,0.309,14.600,29.835,29.983,29.983,29.983,2.285,18.715,0.383,3.306,4.930,1.28%,JJSF,True
3,10026,20160930.0,20160930,20161231,37.072,0.309,14.600,32.784,32.946,32.946,32.946,2.513,20.579,0.383,3.636,5.417,1.26%,JJSF,True
4,10026,20160930.0,20160930,20170131,35.467,0.309,14.600,31.344,31.499,31.499,31.499,2.404,19.688,0.383,3.478,5.179,1.32%,JJSF,True


In [17]:
r = r.drop(drop_columns,axis=1)

In [18]:
r = r.drop(['permno', 'adate', 'qdate', 'yn'],axis=1)

In [19]:
len(r.columns)

12

In [20]:
r.columns

Index(['public_date', 'CAPEI', 'bm', 'evm', 'pe_op_basic', 'pe_op_dil',
       'pe_exi', 'pe_inc', 'ps', 'pcf', 'ptb', 'TICKER'],
      dtype='object')

In [21]:
# datetime transform
r['public_date'] = pd.to_datetime(r['public_date'],format='%Y%m%d')
data['datadate'] = pd.to_datetime(data['datadate'],format='%Y-%m-%d')

In [22]:
# matching up

In [23]:
r.columns

Index(['public_date', 'CAPEI', 'bm', 'evm', 'pe_op_basic', 'pe_op_dil',
       'pe_exi', 'pe_inc', 'ps', 'pcf', 'ptb', 'TICKER'],
      dtype='object')

In [24]:
# columns we want
column_title = ['CAPEI', 'bm', 'evm', 'pe_op_basic', 'pe_op_dil',
                'pe_exi', 'pe_inc', 'ps', 'pcf', 'ptb']

# df to store matched data
match_df = pd.DataFrame(columns = column_title)

# loop through the data
for i in tqdm(range(len(data))):
    tic = data['tic'].iloc[i]
    date = data['datadate'].iloc[i]
    tmp = r[r['TICKER'] == tic] 
    tmp = tmp[date > tmp['public_date']]
    if len(tmp)-1 >= 0:
        tmp = tmp.iloc[len(tmp)-1][column_title]
    else:
        tmp = pd.Series(data= float("NaN"), index=column_title)
        tmp.name = i
    match_df = match_df.append(tmp)

100%|██████████| 255925/255925 [53:20<00:00, 79.96it/s]


In [25]:
match_df

,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,pcf,ptb
48612,-7.819,NaN,-12.471,-6.564,-6.564,-6.244,-6.244,3.7,-8.994,NaN
48612,-7.819,NaN,-12.471,-6.564,-6.564,-6.244,-6.244,3.7,-8.994,NaN
48612,-7.819,NaN,-12.471,-6.564,-6.564,-6.244,-6.244,3.7,-8.994,NaN
48612,-7.819,NaN,-12.471,-6.564,-6.564,-6.244,-6.244,3.7,-8.994,NaN
48612,-7.819,NaN,-12.471,-6.564,-6.564,-6.244,-6.244,3.7,-8.994,NaN
...,...,...,...,...,...,...,...,...,...,...
28981,7.202,0.971,22.733,15.034,15.034,1.608,1.608,2.5,-1.246,0.886
28981,7.202,0.971,22.733,15.034,15.034,1.608,1.608,2.5,-1.246,0.886
28981,7.202,0.971,22.733,15.034,15.034,1.608,1.608,2.5,-1.246,0.886
28981,7.202,0.971,22.733,15.034,15.034,1.608,1.608,2.5,-1.246,0.886


In [26]:
match_df.to_csv("/content/drive/MyDrive/Social Media Group /Data/Test_result/matched_ratios_test.csv")

In [27]:
len(match_df)

255925